In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv("../../../card/train.csv",index_col=0)
test = pd.read_csv("../../../card/test.csv",index_col=0)
sub  = pd.read_csv('../../../card/sample_submission.csv')

In [3]:
train.loc[train.DAYS_EMPLOYED >0,'DAYS_EMPLOYED'] =0
test.loc[test.DAYS_EMPLOYED >0,'DAYS_EMPLOYED'] =0

NumericCol=train.select_dtypes(include=np.number).columns
StrCol=train.select_dtypes(include=object).columns

from sklearn.ensemble import RandomForestClassifier

def checkUniqueObject(train,test):
    target_col = train.select_dtypes(include= object).columns
    
    check =0
    for col in target_col:
        trainU = train[col].unique()
        testU  = test[col].unique()

        if set(trainU) != set(testU):
            print(col,' have not same Unique values')
            check=1
            
    if check ==0:
        print('all columns have Same Unique values')

checkUniqueObject(train,test)

def LabelEn(train,test):
    trainLa = train.copy()
    testLa  = test.copy()
    
    from sklearn.preprocessing import LabelEncoder
    target_col = train.select_dtypes(include= object).columns
    for col in target_col:
        le = LabelEncoder()
        le.fit(trainLa[col])
        trainLa[col] = le.transform(trainLa[col])
        testLa[col] = le.transform(testLa[col])
        
        
    
    return trainLa , testLa

trainLa , testLa = LabelEn(train,test)

x = trainLa.drop('credit',axis=1)
y = trainLa['credit']

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

In [12]:
rf = RandomForestClassifier(random_state=43)

In [13]:
rf.fit(X_train,y_train)

RandomForestClassifier(random_state=43)

In [14]:
from sklearn.metrics import log_loss

In [15]:
clf_probs = rf.predict_proba(X_test)
score = log_loss(y_test, clf_probs)

score

0.9974201226349136

In [16]:
final_Pred = rf.predict_proba(testLa)

In [17]:
sub  = pd.read_csv('../../../card/sample_submission.csv')
sub[['0','1','2']] =final_Pred
sub.to_csv("submission.csv",index=0)